# 0. Install Dependencies

In [1]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

# 1. Test Random Environment with OpenAI Gym

In [1]:
import gym 
import random

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [3]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [6]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:410.0
Episode:2 Score:235.0
Episode:3 Score:155.0
Episode:4 Score:240.0
Episode:5 Score:120.0


# 2. Create a Deep Learning Model with Keras

In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [5]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [11]:
del model

In [23]:
model = build_model(height, width, channels, actions)

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_11 (Dense)             (None, 6)                

# 3. Build Agent with Keras-RL

In [14]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [17]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [25]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [18]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
 1197/10000: episode: 1, duration: 16.708s, episode steps: 1197, steps per second:  72, episode reward: 275.000, mean reward:  0.230 [ 0.000, 30.000], mean action: 2.595 [0.000, 5.000],  loss: 27.396054, mean_q: 11.414030, mean_eps: 0.901135
 1890/10000: episode: 2, duration: 41.929s, episode steps: 693, steps per second:  17, episode reward: 135.000, mean reward:  0.195 [ 0.000, 30.000], mean action: 2.505 [0.000, 5.000],  loss: 1.129293, mean_q: 9.534999, mean_eps: 0.861130
 2441/10000: episode: 3, duration: 34.082s, episode steps: 551, steps per second:  16, episode reward: 140.000, mean reward:  0.254 [ 0.000, 30.000], mean action: 2.566 [0.000, 5.000],  loss: 1.161340, mean_q: 10.541823, mean_eps: 0.805150
 3449/10000: episode: 4, duration: 61.689s, episode steps: 1008, steps per second:  16, episode reward: 320.000, mean reward:  0.317 [ 0.000, 30.000], mean action: 2.569 [0.000, 5.000],  loss: 1.287153, mean_q: 11.357870, mean_eps: 0.734995
 4513/100

In [27]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 90.000, steps: 499
Episode 2: reward: 220.000, steps: 965
Episode 3: reward: 230.000, steps: 1224
Episode 4: reward: 240.000, steps: 715
Episode 5: reward: 635.000, steps: 1267
Episode 6: reward: 205.000, steps: 673
Episode 7: reward: 435.000, steps: 1096
Episode 8: reward: 315.000, steps: 1166
Episode 9: reward: 170.000, steps: 679
Episode 10: reward: 280.000, steps: 871
282.0


# 4. Reloading Agent from Memory

In [21]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

In [22]:
del model, dqn

In [26]:
dqn.load_weights('SavedWeights/1m/dqn_weights.h5f')